## Contenido del proyecto
### EDA
* [0. Importar librerias](#c0)
* [1. Cargar el conjunto de datos](#c1)
* [2. Limpieza de datos](#c2)
    * [2.1 Identificar las variables](#s21)
    * [2.2 Identificar duplicados y valores nulos](#s22)
    * [2.3 s](#s23)
* [3. Análisis Univariante](#c3)
    * [3.1 ](#s31)
    * [3.2 ](#s32) 
    * [3.3 ](#s33)
* [4. Análisis Multivariante](#c4)
    * [4.1](#s41)
    * [4.2 ](#s42)
* [5. Ingeniería de características](#c5)
    * [5.1 ](#s51)
    * [5.2 ](#s52)
    * [5.3 ](#s53)
* [6. Análisis de valores faltantes](#c6)
    * [](#s61)
    * [](#s62)
    * [](#s63)
### Machine Learning 
* [7.1 Decision Making: Which is the best dataset?](#s71)
* [7.2 Model Optimization](#s72)

### 0. Importar librerias

In [11]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 

import json
from pickle import dump

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, Lasso, Ridge, ElasticNet

import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn
warnings.filterwarnings("ignore", category=FutureWarning)
pd.set_option('display.max_columns', None)

### 1. Cargar el conjunto de datos

In [16]:
total_data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/logistic-regression-project-tutorial/main/bank-marketing-campaign-data.csv")
total_data.head()

,"age;""job"";""marital"";""education"";""default"";""housing"";""loan"";""contact"";""month"";""day_of_week"";""duration"";""campaign"";""pdays"";""previous"";""poutcome"";""emp.var.rate"";""cons.price.idx"";""cons.conf.idx"";""euribor3m"";""nr.employed"";""y"""
0,"56;""housemaid"";""married"";""basic.4y"";""no"";""no"";..."
1,"57;""services"";""married"";""high.school"";""unknown..."
2,"37;""services"";""married"";""high.school"";""no"";""ye..."
3,"40;""admin."";""married"";""basic.6y"";""no"";""no"";""no..."
4,"56;""services"";""married"";""high.school"";""no"";""no..."


### 2. Limpieza de datos
#### 2.1 Identificar las variables

- *age.* Edad del cliente (numérico)
- *job.* Tipo de trabajo (categórico)
- *marital.* Estado civil (categórico)
- *education.* Nivel de educación (categórico)
- *default.* ¿Tiene crédito actualmente? (categórico)
- *housing.* ¿Tiene un préstamo de vivienda? (categórico)
- *loan.* ¿Tiene un préstamo personal? (categórico)
- *contact.* Tipo de comunicación de contacto (categórico)
- *month.* Último mes en el que se le ha contactado (categórico)
- *day_of_week.* Último día en el que se le ha contactado (categórico)
- *duration.* Duración del contacto previo en segundos (numérico)
- *campaign.* Número de contactos realizados durante esta campaña al cliente (numérico)
- *pdays.* Número de días que transcurrieron desde la última campaña hasta que fue contactado (numérico)
- *previous.* Número de contactos realizados durante la campaña anterior al cliente (numérico)
- *poutcome.* Resultado de la campaña de marketing anterior (categórico)
- *emp.var.rate.* Tasa de variación del empleo. Indicador trimestral (numérico)
- *cons.price.idx.* Índice de precios al consumidor. Indicador mensual (numérico)
- *cons.conf.idx.* Índice de confianza del consumidor. Indicador mensual (numérico)
- *euribor3m.* Tasa EURIBOR 3 meses. Indicador diario (numérico)
- *nr.employed.* Número de empleados. Indicador trimestral (numérico)
- *y. TARGET.* El cliente contrata un depósito a largo plazo o no (categórico)

#### 2.2 Identificar duplicados y valores nulos

In [13]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 1 columns):
 #   Column                                                                                                                                                                                                                        Non-Null Count  Dtype 
---  ------                                                                                                                                                                                                                        --------------  ----- 
 0   age;"job";"marital";"education";"default";"housing";"loan";"contact";"month";"day_of_week";"duration";"campaign";"pdays";"previous";"poutcome";"emp.var.rate";"cons.price.idx";"cons.conf.idx";"euribor3m";"nr.employed";"y"  41188 non-null  object
dtypes: object(1)
memory usage: 321.9+ KB
